In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pickle
import numpy as np
from pathlib import Path
from tqdm import tqdm
import json

from sklearn import svm, metrics; 
from sklearn.model_selection import train_test_split

from object_detection_fastai.helper.nms_center_distance import non_max_suppression_by_distance
from exact_sync.v1.api.annotations_api import AnnotationsApi

from exact_sync.v1.models import ImageSet, Team, Product, AnnotationType, Image, Annotation, AnnotationMediaFile
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

In [3]:
username = 'marzahl'
password = '****'
host =  "https://exact.cs.fau.de/"
inference_source = Path('D:/Datasets/EIPH WSI/Humane/inference_results_boxes_all_human.p')
inference_target = Path('D:/Datasets/EIPH WSI/Humane/Human_04_035.p')
inference_target_exact = Path('D:/Datasets/EIPH WSI/Humane/Human_label.txt')

In [ ]:
configuration = Configuration()
configuration.username = username
configuration.password = password
configuration.host =  host

client = ApiClient(configuration)
annotations_api = AnnotationsApi(client)

In [ ]:
inputArchive = pickle.load(open(str(inference_source),'rb'))
nms_thresh = 40 
thresh = 0.35

In [ ]:
results = {}

exact_lines = []
for file in inputArchive:
    boxes = np.array(inputArchive[file])

    scales = ((boxes[:, 2] - boxes[:, 0]) / (boxes[:, 3] - boxes[:, 1]))
    boxes = boxes[(scales < 1.1) & (scales > 0.9)]
    annos = non_max_suppression_by_distance(boxes, boxes[:, 5], nms_thresh)
    annos = annos[annos[:, 5] > thresh]

    for anno in annos:
        box_dict = json.dumps({"x1":max(0,anno[0]), "y1": max(0, anno[1]), "x2": anno[2], "y2": anno[3]})


        row = "{0}|{1}|".format(file, int(anno[4])) + box_dict + "\n"
        exact_lines.append(row)

    results[file] = annos

with open(str(inference_target_exact), 'w') as f:
    f.writelines(exact_lines)

pickle.dump(results, open(str(inference_target), "wb" ))